# DC House Price 

#### Data source: https://www.redfin.com/blog/data-center

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Data Import and Data Cleaning
### Importing data and creating dataframes for each Washington DC region
#### All required dataset are downloaded in data folder

In [2]:
# set up name label to match each csv dataset
lst = []
for i in range(1,83):
    location = 'data/'
    ext = 'data_crosstab ({}).csv'.format(i)
    final = location + ext
    lst.append(final)
    # print(final)

# pull datasets from data folder and clean data such as $1,200K, 1.5%, etc. Change data type from object to float
# create a new list: df_list to include all datasets
df_list = []
i = 0
for location in lst:
    df = pd.read_csv(location, encoding='utf-16', sep='\t')
    df = df.add_suffix('_'+str(i))
    # reformat the median sale prices from strings to floats
    #df["Median Sale Price" + "_"+ str(i)] = df["Median Sale Price" + "_"+ str(i)].str.replace("$", "").str.replace(",", "").str.replace("K","000").str.replace("%","").astype(float)
    for j in range(len(df.columns)-2):
        if df[df.columns[j+2]].dtype == 'object':
            df[df.columns[j+2]] = df[df.columns[j+2]].str.replace("$", "").str.replace(",", "").str.replace("K","000").str.replace("%","").astype(float)
        #df[df.columns[j+2]] = df[df.columns[j+2]].str.replace("$", "").str.replace(",", "").str.replace("K","000").str.replace("%","").astype(float)
    df_list.append(df)
    i += 1
# df_list[0].head()

In [ ]:
# review the column names 
df.columns

In [ ]:
# Ensure the data type is appropriate for analysis
df.info()

### Creating dataframe containing median sale prices, Homes Sold MoM and Inventory MoM of each Washington DC region from Feb. 2012 to Oct. 2019 

In [5]:
# Create datasets for median sale prices, Homes Sold MoM and Inventory MoM and save them in data folder


#Creating dataframe containing Median Sale Price of each Washington DC region from Feb. 2012 to Oct. 2019 
final_lst = []
i = 0
for df in df_list:
    final_lst.append(df["Median Sale Price" + "_"+ str(i)][0:93])
    i += 1
median_sale_price = pd.concat(final_lst, axis = 1)
median_sale_price.to_csv('data/Median Sale Price.csv')
#median_sale_price

#Creating dataframe containing Homes Sold MoM of each Washington DC region from Feb. 2012 to Oct. 2019 
final_lst_2 = []
i = 0
for df in df_list:
    final_lst_2.append(df["Homes Sold MoM " + "_"+ str(i)][0:93])
    i += 1
homes_sold_mom = pd.concat(final_lst_2, axis = 1)
homes_sold_mom.to_csv('data/Homes Sold MoM.csv')
#homes_sold_mom

#Creating dataframe containing inventory MoM of each Washington DC region from Feb. 2012 to Oct. 2019 
final_lst_3 = []
i = 0
for df in df_list:
    final_lst_3.append(df["Inventory MoM " + "_"+ str(i)][0:93])
    i += 1
inventory_mom = pd.concat(final_lst_3, axis = 1)
inventory_mom.to_csv('data/Inventory MoM.csv')
#inventory_mom

## Hypothesis Tests and Visualizations

#### Median Sale Price Hypothesis Tests

In [7]:
# median_sale_price is the final dataset(panda.dataframe) with only median sale price for each region. Say n regions
# Create a list combine_list containing any two regions out of n region: list(itertools.combinations(range(A.shape[1]), 2))
# Run a loop to perform hypothesis testings: T test, Wilcoxon test, KS test, Mann-Whitney rank test 

import itertools
from statsmodels.stats.weightstats import ttest_ind as t_test
from scipy.stats import wilcoxon
from scipy.stats import ks_2samp 
from scipy.stats import mannwhitneyu

 
combine_list = list(itertools.combinations(range(median_sale_price.shape[1]), 2))
#statistic, p-value, degree of freedom for two-sample t test
t_stat = []
p_val_t = []
df = []

# statistic and p-value for Wilcoxon test 
non_stat = []
p_val_wc = []

# statistic and p-value for KS test 
ks_stat = []
p_val_ks = []

# statistic and p-value for Mann-Whitney rank test  test 
mu_stat = []
p_val_mu = []

for i in range(len(combine_list)):
    temp1, temp2, temp3 = t_test(median_sale_price.iloc[:,combine_list[i][0]], median_sale_price.iloc[:,combine_list[i][1]], alternative = "two-sided", usevar = "unequal")
    t_stat.append(float(temp1)) 
    p_val_t.append(float(temp2))
    df.append(float(temp3))
    temp4, temp5 = wilcoxon(median_sale_price.iloc[:,combine_list[i][0]], median_sale_price.iloc[:,combine_list[i][1]], alternative = "two-sided", zero_method = "zsplit")
    non_stat.append(float(temp4))
    p_val_wc.append(float(temp5)) 
    temp6, temp7 = ks_2samp(median_sale_price.iloc[:,combine_list[i][0]], median_sale_price.iloc[:,combine_list[i][1]], alternative = "two-sided", mode = 'asymp')
    ks_stat.append(float(temp6))
    p_val_ks.append(float(temp7))
    temp8, temp9 = mannwhitneyu(median_sale_price.iloc[:,combine_list[i][0]], median_sale_price.iloc[:,combine_list[i][1]], alternative = "two-sided")
    mu_stat.append(float(temp8))
    p_val_mu.append(float(temp9))

test_matrix = pd.DataFrame(list(zip(combine_list, t_stat, p_val_t, df, non_stat, p_val_wc, ks_stat, p_val_ks, mu_stat, p_val_mu)), 
               columns =['combinatory', 'Stat_ttest', 'p_ttest', 'df_ttest', 'Stat_wctest', 'p_wctest', 'Stat_kstest', 'p_kstest', 'Stat_mutest', 'p_mutest'])
#test_matrix contains the test results for our median_sale_price hypothesis tests

In [9]:
# Create a function to detect the hypothesis testing results. 0: reject null, 1: fail to reject null
def test_p_value(p = .05, name = 'ttest'):
    name_list = []
    for i in range(len(test_matrix)):
        if test_matrix["p_" + name][i] < p:
            name_list.append(0) 
        else:
            name_list.append(1)   
    test_matrix['index_' + name] = name_list
    
    
test_p_value(p = .05, name = 'ttest')
test_p_value(p = .05, name = 'wctest')
test_p_value(p = .05, name = 'kstest')
test_p_value(p = .05, name = 'mutest')
#test_matrix
#test_matrix.shape

# Save the final testing results for median_sale_price in data folder
test_matrix.to_csv('data/TM_Median Sale Price.csv')

In [ ]:
# Find all regions with similar characteristic based on both Wilcoxon and Mann Whitney tests
index_wc = test_matrix['index_wctest'] == 1
index_mu = test_matrix['index_mutest'] == 1
#test_matrix[index_wc]
#test_matrix[index_mu]
test_matrix[index_wc & index_mu]

In [ ]:
#Find names of matched regions from above cell
df_list[1]['Region_1'][0]
df_list[18]['Region_18'][0]
df_list[35]['Region_35'][0]
df_list[65]['Region_65'][0]

In [17]:
# Set up data time frame
time_frame = pd.date_range('2012-02-01','2019-10-01', 
              freq='MS').strftime("%Y-%b").tolist()

In [18]:
#Create dataset of first 8 regions for visualization purposes, using median_sale_price
df_temp = pd.DataFrame(list(zip(time_frame, median_sale_price['Median Sale Price_0'], median_sale_price['Median Sale Price_1'], median_sale_price['Median Sale Price_2'], median_sale_price['Median Sale Price_3'], median_sale_price['Median Sale Price_4'], median_sale_price['Median Sale Price_5'], median_sale_price['Median Sale Price_6'], median_sale_price['Median Sale Price_7'])), 
               columns =['Time', df_list[0]['Region_0'][0], df_list[1]['Region_1'][0], df_list[2]['Region_2'][0], df_list[3]['Region_3'][0], df_list[4]['Region_4'][0], df_list[5]['Region_5'][0], df_list[6]['Region_6'][0], df_list[7]['Region_7'][0]])

In [ ]:
# plot of first 8 regions, using median_sale_price
sns_plot = sns.lineplot(df_temp['Time'], df_temp.iloc[:,1], label = df_temp.columns[1])
sns_plot = sns.lineplot(df_temp['Time'], df_temp.iloc[:,2], label = df_temp.columns[2])
sns_plot = sns.lineplot(df_temp['Time'], df_temp.iloc[:,3], label = df_temp.columns[3])
sns_plot = sns.lineplot(df_temp['Time'], df_temp.iloc[:,4], label = df_temp.columns[4])
sns_plot = sns.lineplot(df_temp['Time'], df_temp.iloc[:,5], label = df_temp.columns[5])
sns_plot = sns.lineplot(df_temp['Time'], df_temp.iloc[:,6], label = df_temp.columns[6])
sns_plot = sns.lineplot(df_temp['Time'], df_temp.iloc[:,7], label = df_temp.columns[7])
sns_plot = sns.lineplot(df_temp['Time'], df_temp.iloc[:,8], label = df_temp.columns[8])


plt.legend()
plt.title("Time Series - Median House Price",fontsize = 25)
plt.xlabel("Time", fontsize = 15)
plt.ylabel("Median House Price", fontsize = 25)
sns.set(rc={'figure.figsize':(15,10)})
plt.xticks(range(5, 100, 12))
#plt.xlim()
plt.show()
sns_plot.figure.savefig("data_visualizations/Time Series - Median House Price.png")

In [22]:
#Create dataset of 4 matching regions, based on median sale price
df_temp = pd.DataFrame(list(zip(time_frame, median_sale_price['Median Sale Price_1'], median_sale_price['Median Sale Price_18'], median_sale_price['Median Sale Price_35'], median_sale_price['Median Sale Price_65'])), 
               columns =['Time', df_list[1]['Region_1'][0], df_list[18]['Region_18'][0], df_list[35]['Region_35'][0], df_list[65]['Region_65'][0]])
#df_temp1 = df_temp.set_index(df['Time'])

In [ ]:
# plot of 4 matched regions, based on median sale price
sns_plot = sns.lineplot(df_temp['Time'], df_temp.iloc[:,1], label = 'American University Park / Friendship Heights / Tenleytown')
sns_plot = sns.lineplot(df_temp['Time'], df_temp.iloc[:,2], label = 'Chevy Chase-DC')
sns_plot = sns.lineplot(df_temp['Time'], df_temp.iloc[:,3], label = 'Foxhall Village')
sns_plot = sns.lineplot(df_temp['Time'], df_temp.iloc[:,4], label = 'Southeast Chevy Chase')

plt.legend()
plt.title("Time Series - Median House Price",fontsize = 20)
plt.xlabel("Time", fontsize = 20)
plt.ylabel("Median House Price", fontsize = 20)
sns.set(rc={'figure.figsize':(15,10)})
plt.xticks(range(5, 100, 12))
#plt.xlim()
plt.show()
sns_plot.figure.savefig("data_visualizations/Time Series - Median House Price_Group.png")

#### Homes Sold MoM Hypothesis Tests

In [ ]:
# homes_sold_mom is the final dataset(panda.dataframe) with only homes sold MoM for each region. Say n regions
# Create a list combine_list containing any two regions out of n region: list(itertools.combinations(range(A.shape[1]), 2))
# Run a loop to perform hypothesis testings: T test, Wilcoxon test, KS test, Mann-Whitney rank test 

import itertools
from statsmodels.stats.weightstats import ttest_ind as t_test
from scipy.stats import wilcoxon
from scipy.stats import ks_2samp 
from scipy.stats import mannwhitneyu

combine_list = list(itertools.combinations(range(homes_sold_mom.shape[1]), 2))

#statistic, p-value, degree of freedom for two-sample t test
t_stat = []
p_val_t = []
df = []

# statistic and p-value for Wilcoxon test 
non_stat = []
p_val_wc = []

# statistic and p-value for KS test 
ks_stat = []
p_val_ks = []

# statistic and p-value for Mann-Whitney rank test  test 
mu_stat = []
p_val_mu = []

for i in range(len(combine_list)):
    temp1, temp2, temp3 = t_test(homes_sold_mom.iloc[:,combine_list[i][0]], homes_sold_mom.iloc[:,combine_list[i][1]], alternative = "two-sided", usevar = "unequal")
    t_stat.append(float(temp1)) 
    p_val_t.append(float(temp2))
    df.append(float(temp3))
    temp4, temp5 = wilcoxon(homes_sold_mom.iloc[:,combine_list[i][0]], homes_sold_mom.iloc[:,combine_list[i][1]], alternative = "two-sided", zero_method = "zsplit")
    non_stat.append(float(temp4))
    p_val_wc.append(float(temp5)) 
    temp6, temp7 = ks_2samp(homes_sold_mom.iloc[:,combine_list[i][0]], homes_sold_mom.iloc[:,combine_list[i][1]], alternative = "two-sided", mode = 'asymp')
    ks_stat.append(float(temp6))
    p_val_ks.append(float(temp7))
    temp8, temp9 = mannwhitneyu(homes_sold_mom.iloc[:,combine_list[i][0]], homes_sold_mom.iloc[:,combine_list[i][1]], alternative = "two-sided")
    mu_stat.append(float(temp8))
    p_val_mu.append(float(temp9))

test_matrix_2 = pd.DataFrame(list(zip(combine_list, t_stat, p_val_t, df, non_stat, p_val_wc, ks_stat, p_val_ks, mu_stat, p_val_mu)), 
               columns =['combinatory', 'Stat_ttest', 'p_ttest', 'df_ttest', 'Stat_wctest', 'p_wctest', 'Stat_kstest', 'p_kstest', 'Stat_mutest', 'p_mutest'])
#test_matrix_2 contains the test results for our homes_sold_mom hypothesis tests

In [27]:
# Create a function to detect the hypothesis testing results. 0: reject null, 1: fail to reject null
def test_p_value_2(p = .05, name = 'ttest'):
    name_list = []
    for i in range(len(test_matrix_2)):
        if test_matrix_2["p_" + name][i] < p:
            name_list.append(0) 
        else:
            name_list.append(1)   
    test_matrix_2['index_' + name] = name_list
    

test_p_value_2(p = .05, name = 'ttest')
test_p_value_2(p = .05, name = 'wctest')
test_p_value_2(p = .05, name = 'kstest')
test_p_value_2(p = .05, name = 'mutest')
#(test_matrix_2.index_wctest == 1)


# Save the final testing results for homes_sold_mom in data folder
test_matrix_2.to_csv('data/TM_Homes Sold MoM.csv')

In [30]:
#Create dataset of first 8 regions for visualization purposes, using homes_sold_mom
df_temp_2 = pd.DataFrame(list(zip(time_frame, homes_sold_mom['Homes Sold MoM _0'], homes_sold_mom['Homes Sold MoM _1'], homes_sold_mom['Homes Sold MoM _2'], homes_sold_mom['Homes Sold MoM _3'], homes_sold_mom['Homes Sold MoM _4'], homes_sold_mom['Homes Sold MoM _5'], homes_sold_mom['Homes Sold MoM _6'], homes_sold_mom['Homes Sold MoM _7'])), 
               columns =['Time', df_list[0]['Region_0'][0], df_list[1]['Region_1'][0], df_list[2]['Region_2'][0], df_list[3]['Region_3'][0], df_list[4]['Region_4'][0], df_list[5]['Region_5'][0], df_list[6]['Region_6'][0], df_list[7]['Region_7'][0]])

In [ ]:
# plot of first 8 regions, using homes_sold_mom
sns_plot = sns.lineplot(df_temp_2['Time'], df_temp_2.iloc[:,1], label = df_temp_2.columns[1])
sns_plot = sns.lineplot(df_temp_2['Time'], df_temp_2.iloc[:,2], label = df_temp_2.columns[2])
sns_plot = sns.lineplot(df_temp_2['Time'], df_temp_2.iloc[:,3], label = df_temp_2.columns[3])
sns_plot = sns.lineplot(df_temp_2['Time'], df_temp_2.iloc[:,4], label = df_temp_2.columns[4])
sns_plot = sns.lineplot(df_temp_2['Time'], df_temp_2.iloc[:,5], label = df_temp_2.columns[5])
sns_plot = sns.lineplot(df_temp_2['Time'], df_temp_2.iloc[:,6], label = df_temp_2.columns[6])
sns_plot = sns.lineplot(df_temp_2['Time'], df_temp_2.iloc[:,7], label = df_temp_2.columns[7])
sns_plot = sns.lineplot(df_temp_2['Time'], df_temp_2.iloc[:,8], label = df_temp_2.columns[8])


plt.legend()
plt.title("Time Series - Homes Sold Month-over-Month",fontsize = 15)
plt.xlabel("Time", fontsize = 15)
plt.ylabel("Homes Sold Month-over-Month", fontsize = 15)
sns.set(rc={'figure.figsize':(15,10)})
plt.xticks(range(5, 100, 12))
#plt.xlim()
plt.show()
sns_plot.figure.savefig("data_visualizations/Time Series - Homes Sold Month-over-Month.png")

In [ ]:
# Find all regions with similar characteristic based on both Wilcoxon and Mann Whitney tests
index_wc = test_matrix_2['index_wctest'] == 1
index_mu = test_matrix_2['index_mutest'] == 1
#test_matrix[index_wc]
#test_matrix[index_mu]
temp = test_matrix_2[index_wc & index_mu]
temp[81:120]
#temp[121:160]

In [32]:
#Create dataset of 4 matching regions, based on homes_sold_mom
df_temp_2 = pd.DataFrame(list(zip(time_frame, homes_sold_mom['Homes Sold MoM _1'], homes_sold_mom['Homes Sold MoM _18'], homes_sold_mom['Homes Sold MoM _35'], homes_sold_mom['Homes Sold MoM _65'])), 
               columns =['Time', df_list[1]['Region_1'][0], df_list[18]['Region_18'][0], df_list[35]['Region_35'][0], df_list[65]['Region_65'][0]])

In [ ]:
# plot of 4 matching regions, based on homes_sold_mom
sns_plot = sns.lineplot(df_temp_2['Time'], df_temp_2.iloc[:,1], label = 'American University Park / Friendship Heights / Tenleytown')
sns_plot = sns.lineplot(df_temp_2['Time'], df_temp_2.iloc[:,2], label = 'Chevy Chase-DC')
sns_plot = sns.lineplot(df_temp_2['Time'], df_temp_2.iloc[:,3], label = 'Foxhall Village')
sns_plot = sns.lineplot(df_temp_2['Time'], df_temp_2.iloc[:,4], label = 'Southeast Chevy Chase')

plt.legend()
plt.title("Time Series - Homes Sold Month-over-Month",fontsize = 15)
plt.xlabel("Time", fontsize = 15)
plt.ylabel("Homes Sold Month-over-Month", fontsize = 15)
sns.set(rc={'figure.figsize':(15,10)})
plt.xticks(range(5, 100, 12))
#plt.xlim()
plt.show()
sns_plot.figure.savefig("data_visualizations/Time Series - Homes Sold Month-over-Month_Group.png")

#### Inventory MoM Hypothesis Tests

In [34]:
# inventory_mom is the final dataset(panda.dataframe) with only inventory MoM for each region. Say n regions
# Create a list combine_list containing any two regions out of n region: list(itertools.combinations(range(A.shape[1]), 2))
# Run a loop to perform hypothesis testings: T test, Wilcoxon test, KS test, Mann-Whitney rank test 
import itertools
from statsmodels.stats.weightstats import ttest_ind as t_test
from scipy.stats import wilcoxon
from scipy.stats import ks_2samp 
from scipy.stats import mannwhitneyu
combine_list = list(itertools.combinations(range(inventory_mom.shape[1]), 2))
#statistic, p-value, degree of freedom for two-sample t test
t_stat = []
p_val_t = []
df = []
# statistic and p-value for Wilcoxon test 
non_stat = []
p_val_wc = []
# statistic and p-value for KS test 
ks_stat = []
p_val_ks = []
# statistic and p-value for Mann-Whitney rank test  test 
mu_stat = []
p_val_mu = []
for i in range(len(combine_list)):
    temp1, temp2, temp3 = t_test(inventory_mom.iloc[:,combine_list[i][0]], inventory_mom.iloc[:,combine_list[i][1]], alternative = "two-sided", usevar = "unequal")
    t_stat.append(float(temp1)) 
    p_val_t.append(float(temp2))
    df.append(float(temp3))
    temp4, temp5 = wilcoxon(inventory_mom.iloc[:,combine_list[i][0]], inventory_mom.iloc[:,combine_list[i][1]], alternative = "two-sided", zero_method = "zsplit")
    non_stat.append(float(temp4))
    p_val_wc.append(float(temp5)) 
    temp6, temp7 = ks_2samp(inventory_mom.iloc[:,combine_list[i][0]], inventory_mom.iloc[:,combine_list[i][1]], alternative = "two-sided", mode = 'asymp')
    ks_stat.append(float(temp6))
    p_val_ks.append(float(temp7))
    temp8, temp9 = mannwhitneyu(inventory_mom.iloc[:,combine_list[i][0]], inventory_mom.iloc[:,combine_list[i][1]], alternative = "two-sided")
    mu_stat.append(float(temp8))
    p_val_mu.append(float(temp9))
test_matrix_3 = pd.DataFrame(list(zip(combine_list, t_stat, p_val_t, df, non_stat, p_val_wc, ks_stat, p_val_ks, mu_stat, p_val_mu)), 
               columns =['combinatory', 'Stat_ttest', 'p_ttest', 'df_ttest', 'Stat_wctest', 'p_wctest', 'Stat_kstest', 'p_kstest', 'Stat_mutest', 'p_mutest'])
#test_matrix_3 contains the test results for our inventory_mom hypothesis tests

In [36]:
# Create a function to detect the hypothesis testing results. 0: reject null, 1: fail to reject null

def test_p_value_3(p = .05, name = 'ttest'):
    name_list = []
    for i in range(len(test_matrix_3)):
        if test_matrix_3["p_" + name][i] < p:
            name_list.append(0) 
        else:
            name_list.append(1)   
    test_matrix_3['index_' + name] = name_list
test_p_value_3(p = .05, name = 'ttest')
test_p_value_3(p = .05, name = 'wctest')
test_p_value_3(p = .05, name = 'kstest')
test_p_value_3(p = .05, name = 'mutest')
test_matrix_3

#save inventory_mom test matrix to data folder
test_matrix_3.to_csv('data/TM_Inventory MoM.csv')

In [39]:
#Create dataset of first 8 regions for visualization purposes, using inventory_mom
df_temp_3 = pd.DataFrame(list(zip(time_frame, inventory_mom['Inventory MoM _0'], inventory_mom['Inventory MoM _1'], inventory_mom['Inventory MoM _2'], inventory_mom['Inventory MoM _3'], inventory_mom['Inventory MoM _4'], inventory_mom['Inventory MoM _5'], inventory_mom['Inventory MoM _6'], inventory_mom['Inventory MoM _7'])), 
               columns =['Time', df_list[0]['Region_0'][0], df_list[1]['Region_1'][0], df_list[2]['Region_2'][0], df_list[3]['Region_3'][0], df_list[4]['Region_4'][0], df_list[5]['Region_5'][0], df_list[6]['Region_6'][0], df_list[7]['Region_7'][0]])

In [ ]:
# plot of first 8 regions, using inventory_mom
sns_plot = sns.lineplot(df_temp_3['Time'], df_temp_3.iloc[:,1], label = df_temp_3.columns[1])
sns_plot = sns.lineplot(df_temp_3['Time'], df_temp_3.iloc[:,2], label = df_temp_3.columns[2])
sns_plot = sns.lineplot(df_temp_3['Time'], df_temp_3.iloc[:,3], label = df_temp_3.columns[3])
sns_plot = sns.lineplot(df_temp_3['Time'], df_temp_3.iloc[:,4], label = df_temp_3.columns[4])
sns_plot = sns.lineplot(df_temp_3['Time'], df_temp_3.iloc[:,5], label = df_temp_3.columns[5])
sns_plot = sns.lineplot(df_temp_3['Time'], df_temp_3.iloc[:,6], label = df_temp_3.columns[6])
sns_plot = sns.lineplot(df_temp_3['Time'], df_temp_3.iloc[:,7], label = df_temp_3.columns[7])
sns_plot = sns.lineplot(df_temp_3['Time'], df_temp_3.iloc[:,8], label = df_temp_3.columns[8])


plt.legend()
plt.title("Time Series - Inventory Month-over-Month",fontsize = 15)
plt.xlabel("Time", fontsize = 15)
plt.ylabel("Inventory Month-over-Month", fontsize = 15)
sns.set(rc={'figure.figsize':(15,10)})
plt.xticks(range(5, 100, 12))
#plt.xlim()
plt.show()
sns_plot.figure.savefig("data_visualizations/Time Series - Inventory Month-over-Month.png")

In [41]:
#Create dataset of 4 matching regions, based on inventory_mom
df_temp_3 = pd.DataFrame(list(zip(time_frame, inventory_mom['Inventory MoM _1'], inventory_mom['Inventory MoM _18'], inventory_mom['Inventory MoM _35'], inventory_mom['Inventory MoM _65'])), 
               columns =['Time', df_list[1]['Region_1'][0], df_list[18]['Region_18'][0], df_list[35]['Region_35'][0], df_list[65]['Region_65'][0]])

In [ ]:
# plot of 4 matched regions, based on inventory_mom
sns_plot = sns.lineplot(df_temp_3['Time'], df_temp_3.iloc[:,1], label = 'American University Park / Friendship Heights / Tenleytown')
sns_plot = sns.lineplot(df_temp_3['Time'], df_temp_3.iloc[:,2], label = 'Chevy Chase-DC')
sns_plot = sns.lineplot(df_temp_3['Time'], df_temp_3.iloc[:,3], label = 'Foxhall Village')
sns_plot = sns.lineplot(df_temp_3['Time'], df_temp_3.iloc[:,4], label = 'Southeast Chevy Chase')

plt.legend()
plt.title("Time Series - Inventory Month-over-Month",fontsize = 15)
plt.xlabel("Time", fontsize = 15)
plt.ylabel("Inventory Month-over-Month", fontsize = 15)
sns.set(rc={'figure.figsize':(15,10)})
plt.xticks(range(5, 100, 12))
#plt.xlim()
plt.show()
sns_plot.figure.savefig("data_visualizations/Time Series - Inventory Month-over-Month_Group.png")